In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import os


import cv2
import shutil
import numpy as np

from PIL import Image, ImageDraw
from ultralytics import YOLO

from utils.utils import list_file_r, copy_image_to_sub_dir, create_yolo_annotation_from_mask

In [4]:
# can install package here if needed
import sys
package_name = 'pillow' # enter package name here
!{sys.executable} -m pip install {package_name}

In [57]:
# dataset path
image_data_path = 'datasets/CF_simulation/images/'
sample_img = list_file_r(image_data_path)[-1]

mask_data_path = 'datasets/CF_simulation/mask/'
sample_mask = list_file_r(mask_data_path)[-1]

img, msk = cv2.imread(sample_img), cv2.imread(sample_mask)
msk = cv2.cvtColor(msk, cv2.COLOR_BGR2GRAY)

_, im = cv2.threshold(msk, 100, 255, type=cv2.THRESH_BINARY)

black_border = np.zeros((100, 1000) ,dtype='uint8')
im = np.concatenate((black_border, im), axis=0)
#print(im[-1])
#Image.fromarray(im).show()
contours, hierachy = cv2.findContours(im, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
# dataset path
image_data_path = 'datasets/CF_simulation/images/'
sample_img = list_file_r(image_data_path)[-1]

mask_data_path = 'datasets/CF_simulation/mask/'
sample_mask = list_file_r(mask_data_path)[-1]

img, msk = cv2.imread(sample_img), cv2.imread(sample_mask)
msk = cv2.cvtColor(msk, cv2.COLOR_BGR2GRAY)
_, im = cv2.threshold(msk, 100, 255, type=cv2.THRESH_BINARY)
height, width = im.shape
add_border_on_edge = True
if add_border_on_edge:
    border_height = 100
    black_border = np.zeros((border_height, 1000), dtype='uint8')
    #im = np.concatenate((black_border, im), axis=0)
contours, hierachy = cv2.findContours(im, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


#Image.fromarray(img).show()

annotation_text = ''
bbox_text = ''
for contour in contours:
    # b-box
    bbox_text += '0 '
    x,y,w,h = cv2.boundingRect(contour)
    x,y = x+w/2, y+h/2
    x,y,w,h = x/width, y/height, w/width, h/height
    bbox_text += '{} {} {} {}\n' .format(x,y,w,h)
    # mask
    annotation_text += '0'
    for d in range(len(contour)):
        annotation_text += ' '
        #contour[d][0][1] -= border_height
        X, Y = contour[d][0]
        #Y = Y - border_height
        X, Y = X/width, Y/height
        annotation_text += '{} {}'.format(X, Y)
    annotation_text += '\n' # newline
print(annotation_text)
print(bbox_text)

cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
pil_image = Image.fromarray(img) 
pil_image.show() 
cv2.destroyAllWindows() 


In [4]:
add_border_on_edge = False
mask_data_path = 'datasets/CF_simulation/mask/augmented'
annotation_dst = 'datasets/CF_simulation/labels/augmented'
mask_pfs = list_file_r(mask_data_path)
create_yolo_annotation_from_mask(mask=mask_data_path,out=annotation_dst)
if True:
    for pf in mask_pfs:
        msk = cv2.imread(pf)
        height, width, _ = msk.shape
        msk = cv2.cvtColor(msk, cv2.COLOR_BGR2GRAY)
        _, im = cv2.threshold(msk, 100, 255, type=cv2.THRESH_BINARY)
        if add_border_on_edge:
            black_border = [0 for _ in range(1000)]
            im[0], im[-1] = black_border, black_border
        contours, hierachy = cv2.findContours(im, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        annotation_text = ''
        bbox_text = ''
        for contour in contours:
            # b-box
            bbox_text += '0 '
            x,y,w,h = cv2.boundingRect(contour)
            x,y = x+w/2, y+h/2
            x,y,w,h = x/width, y/height, w/width, h/height
            bbox_text += '{} {} {} {}\n' .format(x,y,w,h)
            # mask
            annotation_text += '0'
            for d in range(len(contour)):
                annotation_text += ' '
                #contour[d][0][1] -= border_height
                X, Y = contour[d][0]
                #Y = Y - border_height
                X, Y = X/width, Y/height
                annotation_text += '{} {}'.format(X, Y)
            annotation_text += '\n' # newline

        file_name = os.path.splitext(os.path.split(pf)[-1])[0]
        annotation_fp = os.path.normpath(os.path.join(annotation_dst, file_name)+'.txt')
        
        with open(annotation_fp, 'w') as mask_annotation_file:
            mask_annotation_file.write(annotation_text)
            mask_annotation_file.close()

In [9]:
# randomly select images for train/validation set
if False:
    image_data_path = 'datasets/CF_simulation/images/augmented'
    annotation_path = 'datasets/CF_simulation/labels/augmented'
    if not os.path.exists(image_data_path):
        os.makedirs(image_data_path)
    if not os.path.exists(annotation_path):
        os.makedirs(annotation_path)
    image_pfs = list_file_r(path=image_data_path, extension=['.tif'])

    rng = np.random.default_rng(seed=12345)
    sample_size, validation_ratio = 20000, 0.2

    selected_image_pfs = rng.choice(image_pfs, size=sample_size, replace=False)
    validation_image_pfs = selected_image_pfs[:int(sample_size*validation_ratio)]
    training_image_pfs = selected_image_pfs[int(sample_size*validation_ratio):]


    copy_image_to_sub_dir(training_image_pfs, sub_dir='train') 
    copy_image_to_sub_dir(validation_image_pfs, sub_dir='val')
    

In [2]:
arguments={
    'epochs':20,
    'name':'augmented',
    'imgsz':480,
    #'epochs':20,
}
model = YOLO('yolov8n-seg.pt')
results = model.train(data='train_augmented.yaml', **arguments)

New https://pypi.org/project/ultralytics/8.1.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.0 🚀 Python-3.11.5 torch-2.1.2+cpu CPU (AMD Ryzen 9 5900X 12-Core Processor)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=train_augmented.yaml, epochs=20, time=None, patience=50, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=augmented, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fal

train: Scanning D:\xie\Vision\mcp_yolo\datasets\CF_simulation\labels\augmented\train... 16000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16000/16000 [00:34<00:00, 469.94it/s]


train: New cache created: D:\xie\Vision\mcp_yolo\datasets\CF_simulation\labels\augmented\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\xie\Vision\mcp_yolo\datasets\CF_simulation\labels\augmented\val... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:08<00:00, 485.14it/s]


val: New cache created: D:\xie\Vision\mcp_yolo\datasets\CF_simulation\labels\augmented\val.cache
Plotting labels to runs\segment\augmented\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      17/20         0G     0.3706     0.5845     0.2629     0.8906         77        480: 100%|██████████| 1000/1000 [26:41<00:00,  1.60s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [03:17<00:00,  1.58s/it]

                   all       4000      26495      0.984      0.851      0.889       0.84      0.982      0.844      0.882       0.77



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      18/20         0G      0.386     0.5828     0.2629     0.8854         86        480:   1%|          | 8/1000 [00:12<26:37,  1.61s/it]

In [ ]:
# validate model
model = YOLO('runs/segment/largedata_20epochs_480p_nano/weights/best.pt')
model.val(data='val.yaml', task='detect')

In [26]:
import PIL.ImageOps as ImageOps
#test_result = model('datasets/CF_simulation/images/IMG10_sin_Radius7_4_Density1500_1000_Width50_Reflection_0_Curve1to20.tif')
#print(test_result)
#print(test_result[0].boxes)
model = YOLO('runs/segment/train/weights/best.pt')
test_img_path = 'datasets/CF_simulation/images/IMG10_sin_Radius7_4_Density1500_1000_Width50_Reflection_0_Curve1to20.tif'
test_result = model.predict(test_img_path, show=True, boxes=False)
#res_plotted = test_result[0].plot()
#print(test_result[0].boxes)
#cv2.imshow("result", res_plotted)
mask_seg = test_result[0].masks.data
print(mask_seg.data.shape)
#img = Image.open(test_img_path)
#mask = Image.fromarray(np.uint8(255*mask))
#seg_img = Image.fromarray(np.uint8(mask_seg.data.numpy()[0]))
#seg_img.show()
#img1 = img.paste(seg_img, (0, 0), seg_img)
#img = Image.fromarray(res_plotted)
#img1.show()
import torchvision.transforms as T
im = np.array(Image.new('RGB', (640, 640), (0,0,0)))
for seg in mask_seg:
    current = T.ToPILImage()(seg).convert('RGB')
    #current = ImageOps.invert(current)
    current = np.array(current)
    print(current)
    im += current
    #im.paste(current)
im = Image.fromarray(im)
#im = ImageOps.invert(im)
im.show()

WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

image 1/1 d:\xie\Vision\YOLOv8\datasets\CF_simulation\images\IMG10_sin_Radius7_4_Density1500_1000_Width50_Reflection_0_Curve1to20.tif: 640x640 6 ink traces, 54.0ms
Speed: 3.0ms preprocess, 54.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
torch.Size([6, 640, 640])
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0

In [ ]:
import itertools
for 



results = model.predict(source=xxx, project="xxx", name="xxx")

In [16]:
import torch
torch.cuda.empty_cache()